In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
import asyncio
from pathlib import Path
import pickle
import functools
import inspect
from typing import Callable

import fbdev
from fbdev._utils import get_function_from_py_file
from fbdev.complib import ExecComponent
from fbdev.dev_utils import method_from_py_file

### Test 1

In [ ]:
class FooComponent(ExecComponent):
    @method_from_py_file('./test_func.py')
    async def _execute(self): pass
    
comp_process = FooComponent()
await comp_process._execute()

Hello world


### Test 2

In [ ]:
class Foo:
    @method_from_py_file('./test_func.py')
    def foo(self): pass
    
Foo().foo()

Hello world


### Test 3

We import `my_module/my_submodule/my_subsubmodule.py`, the contents of which are

```python
from ..a_module import foo
import my_module

foo()
```

Here we test that the relative imports work correctly.

In [ ]:
import my_module
from my_module import a_module

Before `my_module.a_module.foo` is called `a_module.has_been_called` should be `False`. The definition of `foo` is

```python
def foo():
    global has_been_called
    print("Hello world again")
    has_been_called = True
```

In [ ]:
assert not a_module.has_been_called

We now define the component and execute it

In [ ]:
class Foo:
    @method_from_py_file('./my_module/my_submodule/my_subsubmodule.py')
    def foo(self): pass
    
Foo().foo()

Hello world again


`a_module.has_been_called` should be `True`. The below tests that the imported `a_module` is actually the same `a_module` within `MyComponent2.execute`.

In [ ]:
assert a_module.has_been_called